In [1]:
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-120k5cbu
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-120k5cbu
  Resolved https://github.com/huggingface/transformers.git to commit 66bc4def9505fa7c7fe4aa7a248c34a026bb552b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install -q av

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
! ls -lrt "/content/drive/MyDrive"

total 5288016
drwx------ 2 root root       4096 Jul  5  2013 'thesis stuff'
-rw------- 1 root root      78414 May 29  2014  alumni_form.pdf
-rw------- 1 root root    1008650 Jul 13  2014  IMG_20140713_090024121.jpg
-rw------- 1 root root      15373 Aug 30  2014 'link script.txt'
-rw------- 1 root root        184 Sep  7  2014 'Market analysis spread sheet.gsheet'
-rw------- 1 root root        168 Jun  6  2015  NAINITAL.gmap
drwx------ 2 root root       4096 Aug 10  2015  Coding
-rw------- 1 root root      47598 Aug  3  2016 '14866279 RAVI KIRAN.pdf'
-rw------- 1 root root      52384 Aug  3  2016 '14866102 RAVI KIRAN.pdf'
drwx------ 2 root root       4096 Aug 30  2016 'home loan documents'
drwx------ 2 root root       4096 Dec 14  2016  Microsemi
drwx------ 2 root root       4096 Jan 16  2017 'Cisma Laptop'
-rw------- 1 root root        184 Jun  1  2017 'Ladakh Trip Plan.gdoc'
drwx------ 2 root root       4096 Jun 22  2017  Ladakh
-rw------- 1 root root        184 Jul 16  2017  Peace_Pro

In [5]:
! ls /content/drive/MyDrive/INFORMS_Data_Challenge_2024/Testing/testing

6749566550967127301.avi  7009670062571408645.avi	    reencoded_6927589342043065605.avi
6756263982178274565.avi  7010379493084876037.avi	    reencoded_6927652656878030085.avi
6766635679066672389.avi  7011082899038489861.avi	    reencoded_6929998188355095813.avi
6768464664985619718.avi  7013705804087905542.avi	    reencoded_6930210371722693893.avi
6769949449730002182.avi  7014135982068993286.avi	    reencoded_6930959141087268101.avi
6772194541882608902.avi  7015257472063524102.avi	    reencoded_6930974243085667589.avi
6775904904947977477.avi  7015672190364306693.avi	    reencoded_6931422030449396997.avi
6779294352037383429.avi  7017840818362846469.avi	    reencoded_6935420539548814598.avi
6785523851078929669.avi  7018481421803195654.avi	    reencoded_6936185551397604613.avi
6791827770302385413.avi  7018571506485579013.avi	    reencoded_6937690334587555078.avi
6791828548135963910.avi  7018602399828839686.avi	    reencoded_6938684575975476481.avi
6792932804775693573.avi  701861223297194727

In [6]:
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")
model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto'
)

preprocessor_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
import av
import numpy as np

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.

    Args:
        container (av.container.input.InputContainer): PyAV container.
        indices (List[int]): List of frame indices to decode.

    Returns:
        np.ndarray: np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [8]:
import os
import subprocess

def reencode_video(input_path, output_path):
    """
    Re-encodes the input video to the specified output path using FFmpeg,
    converting to H.264 video codec and AAC audio codec.

    Args:
        input_path (str): Path to the input video file.
        output_path (str): Path to the output re-encoded video file.

    Returns:
        bool: True if re-encoding was successful, False otherwise.
    """
    command = [
        'ffmpeg', '-i', input_path,
        '-c:v', 'libx264', '-crf', '23', '-preset', 'fast',
        '-c:a', 'aac', '-b:a', '128k', '-strict', 'experimental',
        '-y',  # Overwrite the output file if it exists
        output_path
    ]

    try:
        # Run the FFmpeg command
        subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Successfully re-encoded video: {output_path}")
        return True
    except subprocess.CalledProcessError as e:
        # Capture and display the error message
        print(f"Error during video re-encoding: {e.stderr.decode()}")
        return False

In [9]:

'''
def get_videos_in_folder(folder_path):
    """
    Gets the paths of all video files in the specified folder, counts them,
    and re-encodes each video to a standard format using FFmpeg.

    Parameters:
    - folder_path (str): The path to the folder containing the video files.

    Returns:
    - tuple: A tuple containing the count of successfully re-encoded video files
             and a list of paths to the re-encoded video files.
    """
    # List of common video file extensions
    video_extensions = ('.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv', '.webm')

    # Initialize lists to store file paths
    video_paths = []
    reencoded_paths = []

    try:
        # Iterate through all files in the folder
        for file_name in os.listdir(folder_path):
            # Full path to the file
            file_path = os.path.join(folder_path, file_name)

            # Check if the file has a video extension and is a file
            if file_name.lower().endswith(video_extensions) and os.path.isfile(file_path):
                video_paths.append(file_path)

                # Define output path for re-encoded video
                reencoded_path = os.path.join(folder_path, f"reencoded_{file_name}")

                # Attempt to re-encode the video
                if reencode_video(file_path, reencoded_path):
                    reencoded_paths.append(reencoded_path)

        # Get the count of successfully re-encoded video files
        reencoded_count = len(reencoded_paths)

        print(f"Found {len(video_paths)} video files in the folder.")
        print(f"Successfully re-encoded {reencoded_count} videos.")

    except FileNotFoundError:
        print(f"Error: The folder '{folder_path}' was not found.")
        reencoded_count = 0
        reencoded_paths = []
    except PermissionError:
        print(f"Error: Permission denied for accessing the folder '{folder_path}'.")
        reencoded_count = 0
        reencoded_paths = []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        reencoded_count = 0
        reencoded_paths = []

    return reencoded_count, reencoded_paths

    '''

'\ndef get_videos_in_folder(folder_path):\n    """\n    Gets the paths of all video files in the specified folder, counts them,\n    and re-encodes each video to a standard format using FFmpeg.\n\n    Parameters:\n    - folder_path (str): The path to the folder containing the video files.\n\n    Returns:\n    - tuple: A tuple containing the count of successfully re-encoded video files \n             and a list of paths to the re-encoded video files.\n    """\n    # List of common video file extensions\n    video_extensions = (\'.mp4\', \'.avi\', \'.mov\', \'.mkv\', \'.flv\', \'.wmv\', \'.webm\')\n\n    # Initialize lists to store file paths\n    video_paths = []\n    reencoded_paths = []\n\n    try:\n        # Iterate through all files in the folder\n        for file_name in os.listdir(folder_path):\n            # Full path to the file\n            file_path = os.path.join(folder_path, file_name)\n\n            # Check if the file has a video extension and is a file\n            if fil

In [10]:

def get_videos_in_folder(folder_path):
    """
    Gets the paths of all video files in the specified folder, counts them,
    and re-encodes each video to a standard format using FFmpeg.

    Parameters:
    - folder_path (str): The path to the folder containing the video files.

    Returns:
    - tuple: A tuple containing the count of successfully re-encoded video files
             and a list of paths to the re-encoded video files.
    """
    # List of common video file extensions
    video_extensions = ('.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv', '.webm')

    # Initialize lists to store file paths
    video_paths = []
    reencoded_paths = []

    try:
        # Iterate through all files in the folder
        for file_name in os.listdir(folder_path):
            # Full path to the file
            file_path = os.path.join(folder_path, file_name)

            # Check if the file has a video extension and is a file
            if file_name.lower().startswith("reencoded_") and os.path.isfile(file_path):
                video_paths.append(file_path)

        print(f"Found {len(video_paths)} video files in the folder.")
        print(f"Successfully re-encoded {reencoded_count} videos.")

    except FileNotFoundError:
        print(f"Error: The folder '{folder_path}' was not found.")
        reencoded_count = 0
        reencoded_paths = []
    except PermissionError:
        print(f"Error: Permission denied for accessing the folder '{folder_path}'.")
        reencoded_count = 0
        reencoded_paths = []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        reencoded_count = 0
        reencoded_paths = []

    return len(video_paths), video_paths

In [11]:
# Example usage
folder_path = '/content/drive/MyDrive/INFORMS_Data_Challenge_2024/Testing/testing'  # Replace with your folder path
video_count, video_paths = get_videos_in_folder(folder_path)

print(f"Number of videos in the folder: {video_count}")


Found 258 video files in the folder.
An unexpected error occurred: local variable 'reencoded_count' referenced before assignment
Number of videos in the folder: 258


In [12]:
# Each "content" is a list of dicts and you can add image/video/text modalities
conversation = [
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "Analyze the TikTok video to identify and describe the actions of the creators, such as gestures, facial expressions, and body language. Determine the mood of the creators by recognizing emotions like happiness, excitement, sadness, or anger. Identify specific elements of the video that indicate humor, sarcasm, irony, or other nuanced tones. Extract features such as speech tone, visual cues, timing of actions, and expressions that contribute to the comedic or sarcastic context of the content. describe that?"},
              {"type": "video"},
              ],
      },
]

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

In [13]:
import pandas as pd

In [14]:
total_data = []
for path in video_paths[85:]:
    data = {}
    data['video_path'] = path

    #new_path = path.replace(".avi", "_re_encode.avi")
    #reencode_video(path, new_path)

    # Download video from the hub
    #video_path_1 = new_path
    container = av.open(path)

    # sample uniformly 8 frames from the video (we can sample more for longer videos)
    total_frames = container.streams.video[0].frames
    indices = np.arange(0, total_frames, total_frames / 8).astype(int)
    video = read_video_pyav(container, indices)

    # we still need to call the processor to tokenize the prompt and get pixel_values for videos
    inputs = processor([prompt], videos=[video], padding=True, return_tensors="pt").to(model.device)
    generate_kwargs = {"max_new_tokens": 256, "do_sample": True, "top_p": 0.9}

    output = model.generate(**inputs, **generate_kwargs)
    generated_text = processor.batch_decode(output, skip_special_tokens=True)
    data['generated_text'] = generated_text
    total_data.append(data)

    df = pd.DataFrame(total_data)
    df.to_csv('/content/drive/MyDrive/output_new_prompt.csv', index=False)

Expanding inputs for image/video tokens in LLaVa-NeXT-Video should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.


None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
Expanding inputs for image.video tokens in LLaVa-NeXT-Video should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])
None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])
None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])
None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])
None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])
None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])
None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])
None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])
None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])
None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])
None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])

In [15]:
total_data

[{'video_path': '/content/drive/MyDrive/INFORMS_Data_Challenge_2024/Testing/testing/reencoded_6881235236663938305.avi',
  'generated_text': ['USER: \nAnalyze the TikTok video to identify and describe the actions of the creators, such as gestures, facial expressions, and body language. Determine the mood of the creators by recognizing emotions like happiness, excitement, sadness, or anger. Identify specific elements of the video that indicate humor, sarcasm, irony, or other nuanced tones. Extract features such as speech tone, visual cues, timing of actions, and expressions that contribute to the comedic or sarcastic context of the content. describe that? ASSISTANT: The video shows a group of individuals in an outdoor setting, possibly in a park or similar public area. One man is seen running along a road with several cars visible in the background. He appears to be wearing shorts and is running quite quickly, with a broad smile across his face, indicating that he is enjoying the activit